# Make ESM embs of entire reviewed Unirpot

- HOW TO TRUNCATE
- WHICH MODEL
- WHICH EMB
---

In [ ]:
# Enable autoreload extension
%load_ext autoreload
# Automatically reload all modules (except built-ins) before executing code
%autoreload 2

import sys
import os
os.chdir("/home/gdallagl/myworkdir/ESMSec")
print(f"Working directory successfully changed to: {os.getcwd()}")

import utils.my_functions as mf
import utils.embeddings_functions as my_embs
import utils.dataset as my_dataset
import matplotlib.pyplot as plt
import scanpy as sc
from sklearn.decomposition import PCA


import torch
import random
import numpy as np
import pandas as pd
from transformers import AutoModel, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

print(torch.__version__)

In [ ]:
# Configuration / hyperparameters
config = {
    "SEED": 42,             # Random seed

    "PROTEIN_MAX_LENGTH": 1000, # Max protein length (for ESM2)
    "PRETRAIN_ESM_CACHE_DIR": "/home/gdallagl/myworkdir/data/esm2-models", # ESM2 model cache dir
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu", # Device to use (cuda or cpu)
    "BATCH_SIZE": 16,

    "TYPE_EMB_FOR_CLASSIFICATION": "agg_mean", #"concat(agg_mean, agg_max)", #"contextualized_embs", #"concat(agg_mean, agg_max)",cls, stacked_linearized_all, agg_mean
    "PRETRAIN_ESM_CHECKPOINT_NAME": 
        #"facebook/esm2_t48_15B_UR50D", 
        "facebook/esm2_t36_3B_UR50D",
        # #"facebook/esm2_t33_650M_UR50D", 
        # #"facebook/esm2_t12_35M_UR50D", 
        # #"facebook/esm2_t6_8M_UR50D", 

    "UNIPROT_PATH": "/home/gdallagl/myworkdir/ESMSec/data/UniProt/human_proteome.tsv"
}
config["PRECOMPUTED_EMBS_PATH"] = os.path.join(
    f"/home/gdallagl/myworkdir/ESMSec/data/UniProt/precomputed_embs",
    f"entire_reviewed_uniprot_{config['PRETRAIN_ESM_CHECKPOINT_NAME'].replace('/', '-')}_"
    f"{config['TYPE_EMB_FOR_CLASSIFICATION']}.safetensors"
)
print(config["PRECOMPUTED_EMBS_PATH"])
config["PRECOMPUTED_EMBS_PATH_PROTEIN_NAMES"] = config["PRECOMPUTED_EMBS_PATH"].replace('.safetensors', '_names.json')
config["PRECOMPUTED_EMBS_PATH_ADATA"] = config["PRECOMPUTED_EMBS_PATH"].replace('.safetensors', '.h5ad')


# Initializations
random.seed(config["SEED"])
np.random.seed(config["SEED"])
torch.manual_seed(config["SEED"])
torch.backends.cudnn.benchmark = True 

---

## Instantiate ESM model

In [ ]:
# Load pre-trained ESM model
esm_model = AutoModel.from_pretrained(
    config["PRETRAIN_ESM_CHECKPOINT_NAME"], 
    cache_dir=config["PRETRAIN_ESM_CACHE_DIR"],
    weights_only=False  
).to(config["DEVICE"]) 

# Checj whcih model has been moded by AutoModel.from_pretrained()
print("\nESM model type", type(esm_model), "\n")

# Load relative tokenizer
tokenizer = AutoTokenizer.from_pretrained(config["PRETRAIN_ESM_CHECKPOINT_NAME"])

## Load dataset and Tokenize

In [ ]:
# ALREDY CHECJ PROTEION S(HUMAN, REVIWED, ...)
uniprot_proteome = pd.read_csv(config["UNIPROT_PATH"])

#uniprot_proteome = uniprot_proteome.head(2)

# Remae
uniprot_proteome = uniprot_proteome.rename(columns={"Entry": "protein", "Sequence": "sequence", "Gene Names (primary)": "gene"})

# For Later
num_samples = uniprot_proteome.shape[0]
emb_dim = esm_model.config.hidden_size
print(num_samples, emb_dim)

# ATTENTION:trucnat seqq to enforce max conetxt of EMS
uniprot_proteome["truncated_sequence"] = uniprot_proteome.sequence.apply(my_dataset.truncate_sequence)

# tokenize truncated seqeunces
    # ATTENTION: they a
print("Tokenizing...")
encoded = tokenizer(
    list(uniprot_proteome.truncated_sequence),
    padding='max_length',
    max_length=config["PROTEIN_MAX_LENGTH"],
    truncation=True,
    return_tensors="pt"
)
# add to dict
input_ids_tensor = encoded["input_ids"]          # shape: (N, L)
attention_mask_tensor = encoded["attention_mask"]

#####################

# save all information needed to tothe model
cache_data = {
    'protein': list(uniprot_proteome.protein),
    'sequence': list(uniprot_proteome.sequence),
    'truncated_sequence': list(uniprot_proteome.truncated_sequence),
    'input_ids': input_ids_tensor,
    'attention_mask': attention_mask_tensor,
    'embedding': torch.zeros((num_samples, 1), dtype=torch.float32), # FAKE
    'label': torch.zeros((num_samples, 1), dtype=torch.float32), # FAKE
    'set': torch.zeros((num_samples, 1), dtype=torch.float32), # FAKE
}


## Create embs


In [ ]:
my_dataset.create_uniprot_embs(
    esm_model,
    config, 
    cache_data, 
    num_samples, 
    emb_dim,
    save_embs_path=config["PRECOMPUTED_EMBS_PATH"],
    save_names_path=config["PRECOMPUTED_EMBS_PATH_PROTEIN_NAMES"],
)

# Create AnnData with protein embs

## How to Load Embeddings in Downstream Scripts


In [ ]:
embs, protein_names_selected = my_dataset.load_embs_safetensor(
    precomputed_embs_path=config["PRECOMPUTED_EMBS_PATH"], # path with tensors
    protein_names_path=config["PRECOMPUTED_EMBS_PATH"].replace('.safetensors', '_names.json'), # path with the prtein names of the tenoser
    protein_to_select=None # take all
    )
embs

## Create Protein anndata

X --> toekn IDs of truncated protein

obsm[X_latent]

obsm[X_pca]

uns[vocabulary] --> Id to aa mapping

obs: copy from uniprot_proteim.tsv


In [ ]:
# # Read matrix
# adata = sc.AnnData(embs.cpu().numpy()) # Transpose to get Prot x dims

# # Assign protein anmes names
# adata.obs_names = protein_names_selected
# #adata.var_names = genes.gene.astype(str)

# adata.obsm["input_ids"] = input_ids_tensor.cpu().numpy()
# adata.obsm["attention_mask"] = attention_mask_tensor.cpu().numpy()

# adata.obs = adata.obs.merge(uniprot_proteome, left_index=True, right_on="protein", how="left")
# adata.uns["vocabulary"] = tokenizer.get_vocab()


# print(adata)
# display(adata.obs)

In [ ]:
# Read matrix
adata = sc.AnnData(input_ids_tensor.cpu().numpy()) # Transpose to get Prot x dims

# Assign protein anmes names
adata.obs_names = protein_names_selected
#adata.var_names = genes.gene.astype(str)

adata.obsm["X_all"] = embs.cpu().numpy()
adata.layers["attention_mask"] = attention_mask_tensor.cpu().numpy()

adata.obs = adata.obs.merge(uniprot_proteome, left_index=True, right_on="protein", how="left", validate="1:1")
adata.obs.set_index("protein", drop=False)
adata.uns["vocabulary"] = { #  problem with serialization
    str(k).replace('-', 'DASH').replace('.', 'DOT'): str(v) 
    for k, v in tokenizer.get_vocab().items()
}

print(adata)
display(adata.obs)

## Compute PCA

In [ ]:
# --- PARAMETERS ---
N_PCs = 200                # max number of components to compute
threshold = 0.5            # desired cumulative variance explained (e.g. 50%)

# --- RUN PCA ---
X = adata.obsm["X_all"]  # attention use ESM latntespace
pca = PCA(n_components=N_PCs, random_state=42, svd_solver="arpack")
adata.obsm["X_pca"] = pca.fit_transform(X)

# --- STORE PCA METADATA (like Scanpy does) ---
adata.uns["pca"] = {}
adata.uns["pca"]["variance"] = pca.explained_variance_.copy()
adata.uns["pca"]["variance_ratio"] = pca.explained_variance_ratio_.copy()

# --- COMPUTE CUMULATIVE VARIANCE ---
var_ratio = pca.explained_variance_ratio_
cumulative = np.cumsum(var_ratio)

# --- FIND MINIMUM NUMBER OF PCs REACHING THRESHOLD ---
n_pcs_optimal = np.argmax(cumulative >= threshold) + 1

# --- SUBSET PCA EMBEDDINGS ---
print("Before:", adata.obsm["X_pca"].shape)
adata.obsm["X_pca"] = adata.obsm["X_pca"][:, :n_pcs_optimal]
print("After:", adata.obsm["X_pca"].shape)

print(f"To explain {threshold*100:.0f}% of variance, keep {n_pcs_optimal} PCs.")

# --- PLOT CUMULATIVE VARIANCE ---
plt.figure(figsize=(6,4))
plt.plot(cumulative, marker='o', lw=1)
plt.axhline(y=threshold, color='r', ls='--', label=f'{threshold*100:.0f}% variance')
plt.axvline(x=n_pcs_optimal, color='g', ls='--', label=f'{n_pcs_optimal} PCs')
plt.xlabel('Number of PCs')
plt.ylabel('Cumulative variance explained')
plt.title('Cumulative PCA variance explained')
plt.legend()
plt.show()


## Compute Umaps

In [ ]:
# --- Neighbors ---
print("Calculating Neightbors...")
sc.pp.neighbors(adata, use_rep="X_pca", n_neighbors=10, n_pcs=n_pcs_optimal, key_added="neighbors_pca") # overwirtes previosn grpah
sc.pp.neighbors(adata, use_rep="X_all", n_neighbors=10, key_added="neighbors_all")# Use raw adata.X

# --- UMAPs ---
print("Calculating Umaps...")
sc.tl.umap(adata, random_state=42, neighbors_key="neighbors_pca", key_added="X_umap_pca") # each time overweites X_umap
sc.tl.umap(adata, random_state=42, neighbors_key="neighbors_all", key_added="X_umap_all") # each time overweites X_umap

# ATTENTION:  if those 100 PCs already capture >99% of variance, the neighbor graphs will be nearly identical


fig, axes = my_embs.plot_embeddings(
    adata,
    basis=["X_umap_pca", "X_umap_all"],
    color=[None],
    title=["UMAP (neighbors from pca space)", "UMAP (neighbors from all features)"],
    size=[5],
    #palette=[None, "tab20", "viridis"],
    ncols=2,  # 2 columns, 2 rows
    figsize=(16, 8), 
    legend_loc=['on data']
)
plt.show()

adata 

## Save Adata

In [ ]:
adata.write_h5ad(config["PRECOMPUTED_EMBS_PATH_ADATA"])

---
---